In [9]:
from pulp import *

# Problem definition
prob = LpProblem("TSP", LpMinimize)
departures = ['C', 'N', 'W', 'A']
arrivals = ['C', 'N', 'W', 'A']

# Travel costs between cities
travel_costs = {
    'C': {'N':10, 'W':25, 'A':8},
    'N': {'C':7, 'W':20},
    'W': {'N':19, 'C':26, 'A':8},
    'A': {'W':5, 'C':6}
}

# Create routes that exist in the travel_costs dictionary
routes = [(d, a) for d in departures for a in arrivals if a in travel_costs[d]]

# Define decision variables for each route (binary decision variables)
variables = LpVariable.dicts("Route", routes, 0, 1, LpBinary)

# Objective function: minimize the total travel cost
prob += lpSum([variables[(o, d)] * travel_costs[o][d] for (o, d) in routes]), "sum_cost"

# Constraints: Each city must be visited exactly once
# For arrivals (each city is arrived at exactly once)
for a in arrivals:
    prob += lpSum([variables[(d, a)] for d in departures if (d, a) in variables]) == 1, f"Sum_Arrival_{a}"

# For departures (each city departs exactly once)
for d in departures:
    prob += lpSum([variables[(d, a)] for a in arrivals if (d, a) in variables]) == 1, f"Sum_Departure_{d}"

# Subtour elimination constraints using MTZ formulation
# Helper variables to track the visit order (one less than the number of cities)
u = LpVariable.dicts('u', departures, lowBound=0, upBound=len(departures)-1, cat=LpInteger)

# # Subtour elimination constraints: Ensure no subtours exist
M= len(departures)
for i in departures:
    for j in arrivals:
        if i != j and (i, j) in routes and i!='C':
            prob += u[i]+1 <= u[j]+ M*(1-variables[(i,j)])
#             prob += u[i]- u[j] + len(departures) * variables[(i, j)] <= len(departures) - 1, f"SubtourElimination_{i}_{j}"

# Solve the problem
prob.solve()

# Output results
print(f"Status: {LpStatus[prob.status]}")
for v in prob.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/opc/.pyenv/versions/3.8.0/envs/label/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9b846dd69425416fb74bd7c6a10a6c2e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/9b846dd69425416fb74bd7c6a10a6c2e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 100 RHS
At line 116 BOUNDS
At line 131 ENDATA
Problem MODEL has 15 rows, 14 columns and 41 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 34.5 - 0.00 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 1 fixed, 0 tightened bounds, 3 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 5 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No

In [10]:
variables


{('C', 'N'): Route_('C',_'N'),
 ('C', 'W'): Route_('C',_'W'),
 ('C', 'A'): Route_('C',_'A'),
 ('N', 'C'): Route_('N',_'C'),
 ('N', 'W'): Route_('N',_'W'),
 ('W', 'C'): Route_('W',_'C'),
 ('W', 'N'): Route_('W',_'N'),
 ('W', 'A'): Route_('W',_'A'),
 ('A', 'C'): Route_('A',_'C'),
 ('A', 'W'): Route_('A',_'W')}

In [13]:
prob

TSP:
MINIMIZE
6*Route_('A',_'C') + 5*Route_('A',_'W') + 8*Route_('C',_'A') + 10*Route_('C',_'N') + 25*Route_('C',_'W') + 7*Route_('N',_'C') + 20*Route_('N',_'W') + 8*Route_('W',_'A') + 26*Route_('W',_'C') + 19*Route_('W',_'N') + 0
SUBJECT TO
Sum_Arrival_C: Route_('A',_'C') + Route_('N',_'C') + Route_('W',_'C') = 1

Sum_Arrival_N: Route_('C',_'N') + Route_('W',_'N') = 1

Sum_Arrival_W: Route_('A',_'W') + Route_('C',_'W') + Route_('N',_'W') = 1

Sum_Arrival_A: Route_('C',_'A') + Route_('W',_'A') = 1

Sum_Departure_C: Route_('C',_'A') + Route_('C',_'N') + Route_('C',_'W') = 1

Sum_Departure_N: Route_('N',_'C') + Route_('N',_'W') = 1

Sum_Departure_W: Route_('W',_'A') + Route_('W',_'C') + Route_('W',_'N') = 1

Sum_Departure_A: Route_('A',_'C') + Route_('A',_'W') = 1

_C1: 4 Route_('N',_'C') - u_C + u_N <= 3

_C2: 4 Route_('N',_'W') + u_N - u_W <= 3

_C3: 4 Route_('W',_'C') - u_C + u_W <= 3

_C4: 4 Route_('W',_'N') - u_N + u_W <= 3

_C5: 4 Route_('W',_'A') - u_A + u_W <= 3

_C6: 4 Route_('A